# Loading Billboard Data

In [ ]:
import numpy as np
import pandas as pd
import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/billboardHot100_1999-2019.csv', index_col=False)

In [ ]:
df2 = df.copy()

#Billboard Cleaning and Exploration

In [ ]:
df2.head(1)

,Unnamed: 0,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
0,1,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus


In [ ]:
df2.dtypes

Unnamed: 0           int64
Artists             object
Name                object
Weekly.rank          int64
Peak.position      float64
Weeks.on.chart     float64
Week                object
Date                object
Genre               object
Writing.Credits     object
Lyrics              object
Features            object
dtype: object

In [ ]:
df2['Release_Date'] = df2['Date']
df2 = df2.drop(['Date'], axis=1)

In [ ]:
# convert release date to datetime format
df2['Release_Date'] = pd.to_datetime(df2['Release_Date'])
df2['Week'] = pd.to_datetime(df2['Week'])

In [ ]:
df2.dtypes

Unnamed: 0                  int64
Artists                    object
Name                       object
Weekly.rank                 int64
Peak.position             float64
Weeks.on.chart            float64
Week               datetime64[ns]
Genre                      object
Writing.Credits            object
Lyrics                     object
Features                   object
Release_Date       datetime64[ns]
dtype: object

In [ ]:
# create granular release time variables
df2["Release_Month"] = df2['Release_Date'].dt.month
df2["Release_Day"] = df2['Release_Date'].dt.day
df2["Release_Year"] = df2['Release_Date'].dt.year

In [ ]:
df2.head(1)

,Unnamed: 0,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Genre,Writing.Credits,Lyrics,Features,Release_Date,Release_Month,Release_Day,Release_Year
0,1,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus,2019-04-05,4.0,5.0,2019.0


In [ ]:
df2.dtypes

Unnamed: 0                  int64
Artists                    object
Name                       object
Weekly.rank                 int64
Peak.position             float64
Weeks.on.chart            float64
Week               datetime64[ns]
Genre                      object
Writing.Credits            object
Lyrics                     object
Features                   object
Release_Date       datetime64[ns]
Release_Month             float64
Release_Day               float64
Release_Year              float64
dtype: object

In [ ]:
# Change NaN values for number of weeks on chart to zero
df2['Weeks.on.chart'] = df2['Weeks.on.chart'].fillna(0)

In [ ]:
df2 = df2.drop(columns = 'Unnamed: 0')

In [ ]:
df2.head(2)

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Genre,Writing.Credits,Lyrics,Features,Release_Date,Release_Month,Release_Day,Release_Year
0,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus,2019-04-05,4.0,5.0,2019.0
1,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,0.0,2019-07-06,Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN,2019-06-21,6.0,21.0,2019.0


In [ ]:
#Most recent week is 2019-07-06
df2[df2['Name'] == 'Senorita'].head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Genre,Writing.Credits,Lyrics,Features,Release_Date,Release_Month,Release_Day,Release_Year
1,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,0.0,2019-07-06,Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN,2019-06-21,6.0,21.0,2019.0
76502,Justin Timberlake,Senorita,95,27.0,17.0,2003-11-18,"R&;B,Pop","Chad hugo, Pharrell williams, Justin timberlake",Senorita \nLadies and gentlemen\nIt's my pleas...,NaN,2003-07-08,7.0,8.0,2003.0
76590,Justin Timberlake,Senorita,91,27.0,16.0,2003-11-11,"R&;B,Pop","Chad hugo, Pharrell williams, Justin timberlake",Senorita \nLadies and gentlemen\nIt's my pleas...,NaN,2003-07-08,7.0,8.0,2003.0
76688,Justin Timberlake,Senorita,95,27.0,15.0,2003-11-04,"R&;B,Pop","Chad hugo, Pharrell williams, Justin timberlake",Senorita \nLadies and gentlemen\nIt's my pleas...,NaN,2003-07-08,7.0,8.0,2003.0
76772,Justin Timberlake,Senorita,85,27.0,14.0,2003-10-28,"R&;B,Pop","Chad hugo, Pharrell williams, Justin timberlake",Senorita \nLadies and gentlemen\nIt's my pleas...,NaN,2003-07-08,7.0,8.0,2003.0


#Joins for New Billboard Dataframe

In this section, we create a dataframe that summarizes the most significant aspects of the data. It will contain information of a given song's name and primary artist, the number of weeks it stayed on the chart, and its peak position on the chart.

In [ ]:
df3 = df2[['Artists', 'Name', 'Peak.position']]

In [ ]:
df3['Peak.position'] = df3['Peak.position'].fillna(101)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df3.head(2)

,Artists,Name,Peak.position
0,"Lil Nas,",Old Town Road,1.0
1,"Shawn Mendes, Camila Cabello",Senorita,101.0


In [ ]:
df4 = df3.groupby(by = ['Artists', 'Name']).min()

In [ ]:
df4.head()

Peak.position
Artists             Name                           
"Weird Al" Yankovic Canadian Idiot             82.0
                    Word Crimes                39.0
'N Sync             Bye Bye Bye                 4.0
                    Gone                       11.0
                    It's Gonna Be Me            1.0

In [ ]:
df5 = df2[['Artists', 'Name', 'Weeks.on.chart']]

In [ ]:
df5

,Artists,Name,Weeks.on.chart
0,"Lil Nas,",Old Town Road,7.0
1,"Shawn Mendes, Camila Cabello",Senorita,0.0
2,Billie Eilish,Bad Guy,13.0
3,Khalid,Talk,20.0
4,"Ed Sheeran, Justin Bieber",I Don't Care,7.0
...,...,...,...
97220,Vitamin C,Smile,0.0
97221,Collective Soul,Heavy,20.0
97222,Mary Chapin Carpenter,Almost Home,0.0
97223,Q,Vivrant Thing,0.0


#Construct Final Billboard Dataframe

In this section, we clean and prepare the dataframe from the previous section to make it ready for merging with the spotify data

In [ ]:
df6 = df5.groupby(by = ['Artists', 'Name']).max()

In [ ]:
df6.head()

Weeks.on.chart
Artists             Name                            
"Weird Al" Yankovic Canadian Idiot               3.0
                    Word Crimes                  2.0
'N Sync             Bye Bye Bye                 23.0
                    Gone                        24.0
                    It's Gonna Be Me            25.0

In [ ]:
df6['Peak.position'] = df4['Peak.position']

In [ ]:
df6.head()

Weeks.on.chart  Peak.position
Artists             Name                                           
"Weird Al" Yankovic Canadian Idiot               3.0           82.0
                    Word Crimes                  2.0           39.0
'N Sync             Bye Bye Bye                 23.0            4.0
                    Gone                        24.0           11.0
                    It's Gonna Be Me            25.0            1.0

In [ ]:
df6.reset_index(inplace=True)

In [ ]:
#remove single quotes from artist name
df6['Artists'] = df6['Artists'].map(lambda x: x.replace('\'', ''))

In [ ]:
#remove double quotes from artist name
df6['Artists'] = df6['Artists'].map(lambda x: x.replace('\"', ''))

In [ ]:
df6['Artists'] = df6['Artists'].str.lower()

In [ ]:
df6['Name'] = df6['Name'].str.lower()

In [ ]:
new = df6["Artists"].str.split(",", n = 1, expand = True)

In [ ]:
# define the primary artist as the first artist listed in the artist column
df6["Primary_Artist"]= new[0]

In [ ]:
# create a column that uniquely identifies a song by including both its artist and its name
df6['Song_id'] = df6['Primary_Artist'] + ' ' + df6['Name']

In [ ]:
df6.head()

,Artists,Name,Weeks.on.chart,Peak.position,Primary_Artist,Song_id
0,weird al yankovic,canadian idiot,3.0,82.0,weird al yankovic,weird al yankovic canadian idiot
1,weird al yankovic,word crimes,2.0,39.0,weird al yankovic,weird al yankovic word crimes
2,n sync,bye bye bye,23.0,4.0,n sync,n sync bye bye bye
3,n sync,gone,24.0,11.0,n sync,n sync gone
4,n sync,it's gonna be me,25.0,1.0,n sync,n sync it's gonna be me


In [ ]:
df6.columns

Index(['Artists', 'Name', 'Weeks.on.chart', 'Peak.position', 'Primary_Artist',
       'Song_id'],
      dtype='object')

In [ ]:
df6.shape

(7213, 6)

In [ ]:
df6 = df6[df6['Weeks.on.chart'] != 0]

In [ ]:
df6.shape

(5951, 6)

#Spotify Cleaning and Exploration

In [ ]:
# Read data into dataframe
spotify = pd.read_csv('drive/MyDrive/spotify_data.csv', index_col=False)

In [ ]:
#Data cleaning by removing clean versions of explicit songs (because they are duplicates)

spotify = spotify.sort_values(by=['explicit'], ascending=False)
spotify = spotify.drop_duplicates(subset=['artists', 'name'], keep='first')

In [ ]:
spotify.shape

(156608, 19)

In [ ]:
spotify.head(1)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
24149,0.0785,"['Lil Wayne', 'Takeoff']",0.587,200733,0.635,1,03TzvVWdkVbLT9pzewYiDw,0.0,8,0.175,-6.369,1,I Don't Sleep (ft. Takeoff),64,2020-01-31,0.422,82.517,0.758,2020


Below, the same steps to format the final billboard dataframe are carried out on the spotify dataframe

In [ ]:
spotify['artists'] = spotify['artists'].map(lambda x: x.lstrip('[\'').rstrip('\']'))

In [ ]:
spotify['artists'] = spotify['artists'].map(lambda x: x.replace('\'', ''))

In [ ]:
spotify['artists'] = spotify['artists'].str.lower()

In [ ]:
spotify['name'] = spotify['name'].str.lower()

In [ ]:
spotify['artists'] = spotify['artists'].map(lambda x: x.replace('\"', ''))

In [ ]:
new = spotify["artists"].str.split(",", n = 1, expand = True)

In [ ]:
spotify["primary_artist"]= new[0]

In [ ]:
spotify.shape

(156608, 20)

In [ ]:
spotify['song_id'] = spotify['primary_artist'] + ' ' + spotify['name']

In [ ]:
spotify.head(1)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,primary_artist,song_id
24149,0.0785,"lil wayne, takeoff",0.587,200733,0.635,1,03TzvVWdkVbLT9pzewYiDw,0.0,8,0.175,-6.369,1,i don't sleep (ft. takeoff),64,2020-01-31,0.422,82.517,0.758,2020,lil wayne,lil wayne i don't sleep (ft. takeoff)


In [ ]:
spotify.shape

(156608, 21)

In [ ]:
spotify = spotify[spotify['year'] >= 1999]

In [ ]:
spotify.shape

(40302, 21)

In [ ]:
spotify.columns

Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo',
       'valence', 'year', 'primary_artist', 'song_id'],
      dtype='object')

#Merge Dataframes

In this section, the two dataframes are merged using a jaccard join. This is needed because despite our efforts, it is still possible that songs are listed in the two different datasets under slightly different variations of title and artist name.

In [ ]:
!pip3 install py_stringsimjoin
!pip install python-Levenshtein

     |████████████████████████████████| 1.1MB 6.0MB/s 
     |████████████████████████████████| 665kB 35.6MB/s 
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp36-cp36m-linux_x86_64.whl size=3747804 sha256=e29fc467ddf90669d2c5953e4fe9c351b5fbf967ed28d1a8e2224e0cc1584e36
  Stored in directory: /root/.cache/pip/wheels/58/30/8e/6fa463100ff6e8595b131067ec43a118c607f621ac297e7dad
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp36-cp36m-linux_x86_64.whl size=2054270 sha256=2ce9469c7abc0ffef052faced4d578f3bbe03ba2bd849f5a0f400535222f798f
  Stored in directory: /root/.cache/pip/wheels/be/8c/5d/bcdaf17d0784738b73e856ebc4061cd3f2509753a83dd71058
Successfully built py-stringsimjoin py-stringmatching
     |████████████████████████████████| 51kB 2.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144801 sha256=b536e5dfc6a1114af6db137f2740e04f18995a2a321ca40d4a6e7e65304d5524
  Stored in 

In [ ]:
import py_stringsimjoin as ssj
import py_stringmatching as sm
from Levenshtein import distance
from difflib import SequenceMatcher

In [ ]:
tok_5 = sm.QgramTokenizer(qval=5)

This join is an inner join, as all jaccard joins are. Consequently, this dataframe will need to be merged with the original spotify dataframe on a left join in order to include songs that weren't matched.

In [ ]:
new_df = ssj.jaccard_join(spotify, df6, l_key_attr = 'id', r_key_attr = 'Song_id', l_join_attr = 'song_id', r_join_attr = 'Song_id', tokenizer = tok_5, threshold = 0.6, allow_empty=False, l_out_attrs = [
       'name', 'primary_artist', 'song_id'], r_out_attrs = ['Artists', 'Name', 'Weeks.on.chart', 'Peak.position',
       'Primary_Artist', 'Song_id'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
new_df.head(1)

,_id,l_id,r_Song_id,l_name,l_primary_artist,l_song_id,r_Artists,r_Name,r_Weeks.on.chart,r_Peak.position,r_Primary_Artist,_sim_score
0,0,4JqQWAr47pGEoaMArpA7Z3,weird al yankovic word crimes,word crimes,weird al yankovic,weird al yankovic word crimes,weird al yankovic,word crimes,2.0,39.0,weird al yankovic,1.0


In [ ]:
new_df.shape

(5289, 12)

In [ ]:
# songs that weren't matched are added to the same table as the matched songs
merged_df = spotify.merge(new_df, how='left', left_on='song_id', right_on = 'l_song_id')

In [ ]:
merged_df.shape

(40820, 33)

In [ ]:
merged_df.head(1)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,primary_artist,song_id,_id,l_id,r_Song_id,l_name,l_primary_artist,l_song_id,r_Artists,r_Name,r_Weeks.on.chart,r_Peak.position,r_Primary_Artist,_sim_score
0,0.0785,"lil wayne, takeoff",0.587,200733,0.635,1,03TzvVWdkVbLT9pzewYiDw,0.0,8,0.175,-6.369,1,i don't sleep (ft. takeoff),64,2020-01-31,0.422,82.517,0.758,2020,lil wayne,lil wayne i don't sleep (ft. takeoff),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A binary variable is created indicating whether or not the songs has appeared on the billboard top 100 (1 if it has, 0 if not). This must be done using a custom function and the apply function due to the size of the data involved.

In [ ]:
merged_df['billboard_appearance'] = merged_df['l_primary_artist']

In [ ]:
def categorize(x):
    if type(x) == str:
        x = 1
    else:
        x = 0
    return x

In [ ]:
merged_df['billboard_appearance'] = merged_df['billboard_appearance'].apply(categorize)

In [ ]:
merged_df.head(1)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,primary_artist,song_id,_id,l_id,r_Song_id,l_name,l_primary_artist,l_song_id,r_Artists,r_Name,r_Weeks.on.chart,r_Peak.position,r_Primary_Artist,_sim_score,billboard_appearance
0,0.0785,"lil wayne, takeoff",0.587,200733,0.635,1,03TzvVWdkVbLT9pzewYiDw,0.0,8,0.175,-6.369,1,i don't sleep (ft. takeoff),64,2020-01-31,0.422,82.517,0.758,2020,lil wayne,lil wayne i don't sleep (ft. takeoff),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
merged_df = merged_df.drop(['_id', 'l_id', 'r_Song_id', 'l_primary_artist', 'l_name', 'l_song_id', 'r_Artists', 'r_Name', 'r_Weeks.on.chart', 'r_Peak.position', 'r_Primary_Artist', '_sim_score'], axis=1)

In [ ]:
merged_df.head(1)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,primary_artist,song_id,billboard_appearance
0,0.0785,"lil wayne, takeoff",0.587,200733,0.635,1,03TzvVWdkVbLT9pzewYiDw,0.0,8,0.175,-6.369,1,i don't sleep (ft. takeoff),64,2020-01-31,0.422,82.517,0.758,2020,lil wayne,lil wayne i don't sleep (ft. takeoff),0


In [ ]:
merged_df.to_csv('spotify_billboard.csv')

In [ ]:
spotify.shape

(40302, 21)